In [ ]:
from pybiomart import Dataset
from pandas_plink import read_plink
import pyranges as pr
import pandas as pd
import numpy as np

: 

: 

In [2]:
dataset = Dataset(name='btaurus_gene_ensembl',
                  host='http://www.ensembl.org')

for attribute in dataset.attributes.keys():
    if 'reactome' in attribute:
        print(attribute)

reactome
reactome_gene
reactome_transcript


In [3]:
results = dataset.query(attributes=[
    'ensembl_gene_id',
    'external_gene_name',
    'go_id',
    'reactome'
])

In [4]:
gen_path = '../data/ML/BBB2023_MD'
bim, fam, bed = read_plink(gen_path)

Mapping files: 100%|██████████| 3/3 [00:00<00:00,  4.62it/s]
/home/jhonatan/Documents/Doutorado/projects/Snp_models/bio_informed_DNN/venv39_bioDNN/lib/python3.13/site-packages/pandera/_pandas_deprecated.py:146: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [32]:
bim

,chrom,snp,cm,pos,a0,a1,i
0,1,BTA-101873,0.0,776231,G,A,0
1,1,ARS-BFGL-NGS-16466,0.0,907810,T,C,1
2,1,ARS-BFGL-NGS-105096,0.0,993060,G,A,2
3,1,BES1_Contig627_1906,0.0,1032564,A,C,3
4,1,ARS-BFGL-NGS-98142,0.0,1110393,T,C,4
...,...,...,...,...,...,...,...
36299,29,ARS-BFGL-NGS-16401,0.0,50416855,A,G,36299
36300,29,ARS-BFGL-NGS-3284,0.0,50522805,A,G,36300
36301,29,BTA-66351,0.0,50548328,G,A,36301
36302,29,ARS-BFGL-NGS-91027,0.0,50595638,T,C,36302


In [81]:
snps_pos = bim[['snp','chrom', 'pos']]

snps_pos.rename(columns={'snp':'title', 'pos': 'End', 'chrom': 'Chromosome'}, inplace=True)
snps_pos['Start'] = snps_pos['End']-1

# snps_gr = pr.PyRanges(
#     chromosomes = snps_pos["chrom"],
#     starts = snps_pos["pos"] -1,
#     ends=snps_pos["pos"],
# )

snps_gr = pr.PyRanges(snps_pos)
snps_gr

/tmp/ipykernel_92511/1931831179.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_pos.rename(columns={'snp':'title', 'pos': 'End', 'chrom': 'Chromosome'}, inplace=True)
/tmp/ipykernel_92511/1931831179.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_pos['Start'] = snps_pos['End']-1


,title,Chromosome,End,Start
0,BTA-101873,1,776231,776230
1,ARS-BFGL-NGS-16466,1,907810,907809
2,ARS-BFGL-NGS-105096,1,993060,993059
3,BES1_Contig627_1906,1,1032564,1032563
4,ARS-BFGL-NGS-98142,1,1110393,1110392
...,...,...,...,...
36299,ARS-BFGL-NGS-16401,29,50416855,50416854
36300,ARS-BFGL-NGS-3284,29,50522805,50522804
36301,BTA-66351,29,50548328,50548327
36302,ARS-BFGL-NGS-91027,29,50595638,50595637


In [90]:
genes = pr.read_gtf("../data/Bos_taurus.UMD3.1.90.gtf")

genes = genes[genes.Feature == "gene"]

overlap = snps_gr.join(genes)

join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


In [91]:
overlap

,title,Chromosome,End,Start,Source,Feature,Start_b,End_b,Score,Strand,...,transcript_id,transcript_version,transcript_name,transcript_source,transcript_biotype,exon_number,exon_id,exon_version,protein_id,protein_version
0,ARS-BFGL-NGS-66449,1,1204825,1204824,ensembl,gene,1189227,1222846,.,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hapmap53766-ss46526150,1,2082139,2082138,ensembl,gene,2067035,2099875,.,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARS-BFGL-NGS-62826,1,2158691,2158690,ensembl,gene,2109218,2196114,.,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARS-BFGL-NGS-39992,1,2391777,2391776,ensembl,gene,2382872,2459950,.,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BTB-00001612,1,3492221,3492220,ensembl,gene,3483532,3721930,.,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11290,ARS-BFGL-NGS-61470,29,48363525,48363524,ensembl,gene,48217043,48378574,.,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11291,ARS-BFGL-NGS-25479,29,49335967,49335966,ensembl,gene,49332571,49351638,.,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11292,ARS-BFGL-NGS-116005,29,49910007,49910006,ensembl,gene,49879042,49926003,.,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11293,ARS-BFGL-NGS-106356,29,50260533,50260532,ensembl,gene,50238210,50276209,.,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
overlap = overlap[['title','Chromosome', 'Start', 'End', 'Source', 'Feature', 'Start_b', 'End_b',
       'Score', 'Strand', 'Frame', 'gene_id', 'gene_version', 'gene_name',
       'gene_source', 'gene_biotype']]

print(overlap.columns)

Index(['title', 'Chromosome', 'End', 'Start', 'Source', 'Feature', 'Start_b',
       'End_b', 'Score', 'Strand', 'Frame', 'gene_id', 'gene_version',
       'gene_name', 'gene_source', 'gene_biotype'],
      dtype='object')


In [93]:
overlap.df.head()

,title,Chromosome,End,Start,Source,Feature,Start_b,End_b,Score,Strand,Frame,gene_id,gene_version,gene_name,gene_source,gene_biotype
0,ARS-BFGL-NGS-66449,1,1204825,1204824,ensembl,gene,1189227,1222846,.,+,.,ENSBTAG00000008490,5,CRYZL1,ensembl,protein_coding
1,Hapmap53766-ss46526150,1,2082139,2082138,ensembl,gene,2067035,2099875,.,+,.,ENSBTAG00000003064,5,PAXBP1,ensembl,protein_coding
2,ARS-BFGL-NGS-62826,1,2158691,2158690,ensembl,gene,2109218,2196114,.,+,.,ENSBTAG00000003063,5,SYNJ1,ensembl,protein_coding
3,ARS-BFGL-NGS-39992,1,2391777,2391776,ensembl,gene,2382872,2459950,.,+,.,ENSBTAG00000012412,4,URB1,ensembl,protein_coding
4,BTB-00001612,1,3492221,3492220,ensembl,gene,3483532,3721930,.,+,.,ENSBTAG00000017839,5,TIAM1,ensembl,protein_coding


In [141]:
from gprofiler.gprofiler import GProfiler

gp = GProfiler(return_dataframe=True)

# List of unique gene symbols
gene_list = overlap.df['gene_name'].dropna().unique().tolist()

# Query for pathways
gp = GProfiler(return_dataframe=True)

results = gp.profile(
    organism='btaurus',
    query=gene_list,
    sources=['KEGG', 'GO:BP', 'REAC'],  # KEGG, GO biological process, Reactome
    no_evidences=False
)

# View results
print(results.head())

  source      native                                name       p_value  \
0  GO:BP  GO:0065007               biological regulation  7.361622e-80   
1  GO:BP  GO:0050789    regulation of biological process  3.232581e-71   
2  GO:BP  GO:0048856    anatomical structure development  2.762056e-69   
3  GO:BP  GO:0032502               developmental process  9.587048e-67   
4  GO:BP  GO:0007275  multicellular organism development  2.003839e-60   

   significant                                        description  term_size  \
0         True  "Any process that modulates a measurable attri...       8335   
1         True  "Any process that modulates the frequency, rat...       8113   
2         True  "The biological process whose specific outcome...       3832   
3         True  "A biological process whose specific outcome i...       4098   
4         True  "The biological process whose specific outcome...       3013   

   query_size  intersection_size  effective_domain_size  precision    reca

In [ ]:
results['evidences']

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,GO:BP,GO:0065007,biological regulation,7.361622e-80,True,"""Any process that modulates a measurable attri...",8335,3683,2532,15111,0.687483,0.303779,query_1,[GO:0008150],"[PAXBP1, SYNJ1, TIAM1, APP, SAMSN1, EPHA3, ARL...","[[IBA, IEA], [IEA], [IDA, IEA], [IEA], [IBA], ..."
1,GO:BP,GO:0050789,regulation of biological process,3.232581e-71,True,"""Any process that modulates the frequency, rat...",8113,3683,2453,15111,0.666033,0.302354,query_1,"[GO:0008150, GO:0065007]","[PAXBP1, SYNJ1, TIAM1, APP, SAMSN1, EPHA3, ARL...","[[IBA, IEA], [IEA], [IDA, IEA], [IEA], [IBA], ..."
2,GO:BP,GO:0048856,anatomical structure development,2.762056e-69,True,"""The biological process whose specific outcome...",3832,3683,1358,15111,0.368721,0.354384,query_1,[GO:0032502],"[TIAM1, APP, EPHA3, ARL13B, COL8A1, TBC1D23, A...","[[IDA], [IBA, IEA], [IEA], [IEA], [IEA], [IMP]..."
3,GO:BP,GO:0032502,developmental process,9.587048e-67,True,"""A biological process whose specific outcome i...",4098,3683,1423,15111,0.386370,0.347243,query_1,[GO:0008150],"[TIAM1, APP, EPHA3, ARL13B, COL8A1, TBC1D23, A...","[[IDA], [IBA, IEA], [IEA], [IEA], [IEA], [IMP]..."
4,GO:BP,GO:0007275,multicellular organism development,2.003839e-60,True,"""The biological process whose specific outcome...",3013,3683,1101,15111,0.298941,0.365417,query_1,"[GO:0032501, GO:0048856]","[APP, EPHA3, ARL13B, COL8A1, TBC1D23, ALCAM, B...","[[IBA, IEA], [IEA], [IEA], [IEA], [IMP], [IEA]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,GO:BP,GO:0021915,neural tube development,4.659668e-02,True,"""The process whose specific outcome is the pro...",124,3683,51,15111,0.013847,0.411290,query_1,"[GO:0007399, GO:0035295, GO:0043009, GO:0060429]","[ARL13B, DZIP1L, IFT57, LRP2, GLI2, LUZP1, PHA...","[[IEA], [IGI], [IEA], [IEA], [IEA], [IBA, IEA]..."
587,GO:BP,GO:0021954,central nervous system neuron development,4.768364e-02,True,"""The process whose specific outcome is the pro...",70,3683,33,15111,0.008960,0.471429,query_1,"[GO:0021953, GO:0048666]","[ADARB1, ROBO1, EPHB3, EPHB1, GLI2, SLC4A10, E...","[[IMP], [IDA], [IDA], [IEA], [IEA], [IMP], [IE..."
588,GO:BP,GO:1903131,mononuclear cell differentiation,4.897589e-02,True,"""The process in which a relatively unspecializ...",347,3683,118,15111,0.032039,0.340058,query_1,[GO:0002521],"[APP, TRPM2, PLCL2, RABL3, HCLS1, ITGB6, CD28,...","[[IEA], [IEA], [IEA], [IEA], [IMP], [IEA], [ID..."
589,GO:BP,GO:0007611,learning or memory,4.899623e-02,True,"""The acquisition and processing of information...",159,3683,62,15111,0.016834,0.389937,query_1,"[GO:0007610, GO:0050890]","[SYNJ1, APP, MME, CLSTN2, CREB1, TANC1, LMX1A,...","[[IEA], [IEA], [IEA], [IMP], [IMP], [IEA], [IE..."


In [139]:
results['query'].value_counts()

query
query_1    591
Name: count, dtype: int64

### Constructing the neural network using information from the genes and pathways on the structure

In [96]:
overlap

,title,Chromosome,End,Start,Source,Feature,Start_b,End_b,Score,Strand,Frame,gene_id,gene_version,gene_name,gene_source,gene_biotype
0,ARS-BFGL-NGS-66449,1,1204825,1204824,ensembl,gene,1189227,1222846,.,+,.,ENSBTAG00000008490,5,CRYZL1,ensembl,protein_coding
1,Hapmap53766-ss46526150,1,2082139,2082138,ensembl,gene,2067035,2099875,.,+,.,ENSBTAG00000003064,5,PAXBP1,ensembl,protein_coding
2,ARS-BFGL-NGS-62826,1,2158691,2158690,ensembl,gene,2109218,2196114,.,+,.,ENSBTAG00000003063,5,SYNJ1,ensembl,protein_coding
3,ARS-BFGL-NGS-39992,1,2391777,2391776,ensembl,gene,2382872,2459950,.,+,.,ENSBTAG00000012412,4,URB1,ensembl,protein_coding
4,BTB-00001612,1,3492221,3492220,ensembl,gene,3483532,3721930,.,+,.,ENSBTAG00000017839,5,TIAM1,ensembl,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11290,ARS-BFGL-NGS-61470,29,48363525,48363524,ensembl,gene,48217043,48378574,.,-,.,ENSBTAG00000003171,6,SHANK2,ensembl,protein_coding
11291,ARS-BFGL-NGS-25479,29,49335967,49335966,ensembl,gene,49332571,49351638,.,-,.,ENSBTAG00000012742,4,SLC22A18,ensembl,protein_coding
11292,ARS-BFGL-NGS-116005,29,49910007,49910006,ensembl,gene,49879042,49926003,.,-,.,ENSBTAG00000002702,5,TSPAN32,ensembl,protein_coding
11293,ARS-BFGL-NGS-106356,29,50260533,50260532,ensembl,gene,50238210,50276209,.,-,.,ENSBTAG00000010447,3,LSP1,ensembl,protein_coding


In [ ]:
snp_list = overlap.df['title'].unique().tolist()
gene_list = overlap.df['gene_name'].unique().tolist()

n_snps = len(snp_list)
n_genes = len(gene_list)

# Map SNP/Gene to index
snp_idx = {snp:i for i,snp in enumerate(snp_list)}
gene_idx = {gene:i for i,gene in enumerate(gene_list)}

# Initialize mask
mask_snp_gene = np.zeros((n_genes, n_snps), dtype=np.float32)

for _, row in overlap.df.iterrows():
    i = gene_idx[row['gene_name']]
    j = snp_idx[row['title']]
    mask_snp_gene[i,j] = 1.0